In [100]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

instances_size = 'medium_instances'
results_dir = f'MIPLIB_output/new_batch_output/{instances_size}'
# Read all json files in the results directory
file_names = [f for f in listdir(results_dir) if f.endswith('.json')]
instances = dict()
df_list = []
instances_size=instances_size.replace('_1','')
for file in file_names:
    # if not "k0" in file:
        instance_name = file.split('.')[0]
        with open(f'{results_dir}/{file}', 'r') as f:
            data = json.load(f)

        # data to dataframe
        df = pd.DataFrame(data)

        # change "instance" column to name of file (last after /)
        df['instance'] = df['instance'].apply(lambda x: x.split('/')[-1]).apply(lambda x: x.replace('.mps.gz', ''))
        
        # Time correction: If it has more than 1 row, and the -2 row has value "TERMINATION_REASON_TIME_LIMIT" on column "termination_reason", 
        # then remove the last row
        if len(df) > 1 and df.iloc[-2]['termination_reason'] == 'TERMINATION_REASON_TIME_LIMIT':
            # discount the blackbox_time of last row to the total_time column of every row
            df['total_time'] = df['total_time'] - df.iloc[-1]['blackbox_time']
            df = df.iloc[:-1]

        # KKT: norm of the three "relative" metrics
        df['KKT'] = np.sqrt(df['relative_l_inf_primal_residual']**2 + df['relative_l_inf_dual_residual']**2 + df['relative_optimality_gap']**2)    

        # success: if termination_reason of the last row is "TERMINATION_REASON_OPTIMAL", then success = 1, else 0
        df['success'] = 0
        if df.iloc[-1]['termination_reason'] == 'TERMINATION_REASON_OPTIMAL':
            df.loc[:, 'success'] = 1

        # IR: if 'max_iter' is 0, then it is a IR
        df['IR'] = 0
        if df.iloc[0]['max_iter'] > 0:
            df.loc[:, 'IR'] = 1

        # If it is not a IR, then set alpha to 0
        if df.iloc[0]['max_iter'] == 0:
            df['alpha'] = 0

        # add to list
        for row in df.iterrows():
            # print(row[2])
            # row = row[1]
            # row series to list
            row = row[1].tolist()
            df_list.append(pd.DataFrame([row], columns=df.columns))

# Concatenate all dataframes
df = pd.concat(df_list)
# Change column names
col_names_dict = {
    'last_iteration': 'k_iters',
    'total_time': 'time',
    'D1_condition_number': 'D1_maxcn',
    'D2_condition_number': 'D2_maxcn',
}
df.rename(columns=col_names_dict, inplace=True)

# display(df.sample(10))
# Sort by instance, alpha and max_iter
# df.sort_values(by=['instance'], inplace=True)
# Group by instance, alpha and max_iter
grouped = df.groupby(['instance', 'IR', 'alpha'])
# columns to get the min value
tail_cols = ['KKT', 'success', 'time', 'k_iters']
# columns to get the max value
max_cols = ['D1_maxcn', 'D2_maxcn']
grouped = grouped.agg({**{col: 'min' for col in tail_cols}, **{col: 'max' for col in max_cols}})

# Round to 3 decimals time, and condition numbers columns
# grouped['time'] = grouped['time'].apply(lambda x: round(x, 3))
# grouped['D1_maxcn'] = grouped['D1_maxcn'].apply(lambda x: round(x, 3))
# grouped['D2_maxcn'] = grouped['D2_maxcn'].apply(lambda x: round(x, 3))
display(grouped)

# Save grouped dataframe to csv
grouped.to_csv(f'MIPLIB_output/batch_output/{instances_size}_600.csv')


KKT  success        time  k_iters  \
instance            IR alpha                                                
ds-big              0  0.0000  1.439063e-05        0  628.069005        0   
                    1  0.0001  1.185068e-05        0  604.573934        2   
                       1.0010  3.523814e-06        0  604.075764        2   
                       1.0100  1.217117e-04        0  604.951548        2   
                       1.1000  3.439871e-04        0  604.645659        2   
                       1.5000  6.041658e-06        0  604.593942        2   
graph40-80-1rand    0  0.0000  2.862073e-09        1   15.432341        0   
                    1  1.0010  1.237947e-11        1   34.904187        1   
                       1.0100  1.503098e-11        1   34.777784        1   
                       1.1000  1.532925e-11        1   35.099082        1   
                       1.5000  2.728199e-12        1   34.700990        1   
neos-4647030-tutaki 0  0.0000  3.424628e-08        1   26.017920        0   
                    1  1.0010  2.096163e-13        1   27.893211        1   
                       1.0100  1.301761e-12        1   28.494018        1   
                       1.1000  3.954114e-14        1   27.843814        1   
                       1.5000  1.858781e-13        1   29.507241        1   
neos-5049753-cuanza 1  1.0010  7.113425e-10        1   10.267346        1   
                       1.0100  7.852431e-10        1   10.157120        1   
                       1.1000  5.766476e-10        1   10.347971        1   
                       1.5000  3.863232e-10        1   10.260639        1   
neos-5129192-manaia 0  0.0000  2.185556e-08        1   10.530591        0   
                    1  1.0010  2.372368e-08        0  604.940340        3   
                       1.0100  1.020869e-08        0  605.157095        3   
                       1.1000  2.311067e-08        0  604.989194        3   
                       1.5000  3.528028e-08        0  605.066210        3   
seqsolve1           1  1.0010  5.702600e-08        0  602.997060        2   
                       1.0100  5.831795e-08        0  602.995156        2   
                       1.1000  5.541323e-08        0  603.021797        2   

                               D1_maxcn  D2_maxcn  
instance            IR alpha                       
ds-big              0  0.0000       1.0       1.0  
                    1  0.0001       1.0       1.0  
                       1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0  
                       1.5000       1.0       1.0  
graph40-80-1rand    0  0.0000       1.0       1.0  
                    1  1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0  
                       1.5000       1.0       1.0  
neos-4647030-tutaki 0  0.0000       1.0       1.0  
                    1  1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0  
                       1.5000       1.0       1.0  
neos-5049753-cuanza 1  1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0  
                       1.5000       1.0       1.0  
neos-5129192-manaia 0  0.0000       1.0       1.0  
                    1  1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0  
                       1.5000       1.0       1.0  
seqsolve1           1  1.0010       1.0       1.0  
                       1.0100       1.0       1.0  
                       1.1000       1.0       1.0